In [1]:
import asyncio
import nest_asyncio
nest_asyncio.apply()
from concurrent.futures import ThreadPoolExecutor
from random import random
from typing import List

from TM1py import TM1Service
from TM1py.Utils import Utils

async def transfer_async(tm1_src: TM1Service, tm1_tgr: TM1Service,cube_src:str,cube_tgr:str,leaves:List):
    loop = asyncio.get_event_loop()
    
    with ThreadPoolExecutor(10) as executor:
        #futures = [loop.run_in_executor(executor, transfer, tm1_src, tm1_tgr,cube_src, cube_tgr,month_leaf) for month_leaf in leaves]
        futures=[]
        for month_leaf in leaves:
            f=loop.run_in_executor(executor, transfer, tm1_src, tm1_tgr,cube_src, cube_tgr,month_leaf)
            futures.append(f)
        for future in futures:
            try:
                await future
            except:
                print(future)
                continue
    

def transfer(tm1_src: TM1Service, tm1_tgr: TM1Service,cube_src:str,cube_tgr:str,month_leaf:str):
    
    mdx='''
    SELECT 
    {[Month].[Month].['''+month_leaf+''']} ON COLUMNS , 
    {TM1FilterByLevel({TM1SubSetAll([Location])}, 0)} PROPERTIES [Location].[Location].[Full Description]  ON ROWS 
    FROM ['''+cube_src+'''] 
      WHERE 
          (
           [Year].[Year].[2012/2013],
           [Measure_Sales].[Measure_Sales].[Volume],
           [Data Source].[Data Source].[Input],
           [Business Unit].[Business Unit].[81],
           [Version].[Version].[Actual]
          )

         '''
    #元组字典
    cells=tm1_src.cubes.cells.execute_mdx(mdx,skip_rule_derived_cells=True,skip_zeros=True)
    df = Utils.build_pandas_dataframe_from_cellset(cells)
    #维度收拢
    df_less_dim=df.reset_index().drop(['Business Unit','Product'],axis=1)

    #Clear data
    mdx_tgr='''
             SELECT 
               {[Month].[Month].['''+month_leaf+''']} 
              ON COLUMNS , 
               {[Location].[Location].Members} 
               PROPERTIES [Location].[Location].[Full Description]  ON ROWS 
              FROM ['''+cube_tgr+'''] 
                WHERE 
                  (
                   [Year].[Year].[2012/2013],
                   [Version].[Version].[Actual],
                   [Data Source].[Data Source].[Input],
                   [Measure_Sales].[Measure_Sales].[Volume]
                  )   
            
            
                '''
    tm1_tgr.cells.clear_with_mdx(cube_tgr,mdx_tgr)
    #Writing to tgr Cube
    tm1_tgr.cells.write_dataframe(
        cube_name=cube_tgr,
        data=df_less_dim,
        dimensions=['Year','Month','Version','Location','Data Source','Measure_Sales'],
        increment=False,
        deactivate_transaction_log=True,
        reactivate_transaction_log=True,
        sandbox_name=None,
        use_ti=True,
        skip_non_updateable=True,
        use_changeset=False)
    

In [2]:
with TM1Service(address="localhost", port=30012, ssl=False, user="admin", password="") as tm1:
    leaves = tm1.elements.get_leaf_element_names("Month", "Month")
    #leaves=['Jul','Aug','Sep','Oct','Nov','Dec','Jan','Feb','Mar','Apr','May','Jun']
    asyncio.run(transfer_async(tm1, tm1, 'Sales','Sales less product and business unit',leaves))

<Future finished exception=ValueError('Must pass non-zero number of levels/codes',)>
<Future finished exception=ValueError('Must pass non-zero number of levels/codes',)>
<Future finished exception=ValueError('Must pass non-zero number of levels/codes',)>
<Future finished exception=ValueError('Must pass non-zero number of levels/codes',)>
<Future finished exception=ValueError('Must pass non-zero number of levels/codes',)>


In [193]:
with TM1Service(address="localhost", port=30012, ssl=False, user="admin", password="") as tm1:
    transfer(tm1,tm1,'Sales','Sales less product and business unit','Jul')